<a href="https://colab.research.google.com/github/sn0rkmaiden/control_theory/blob/main/Control_theory_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
from sympy import *
from numpy import linalg
from scipy.linalg import expm
import copy

# Program #1

In [42]:
n = 2
P = np.array([[1, 1], [0, 1]])
Q = np.array([[0], [1]])
f = np.array([[0], [0]])
x0 = np.array([[0], [0]])
x1 = np.array([[1], [1]])
T = 1

✅ Check if system is controllable

In [43]:
def isControllable(P, Q, n):
  components = []
  components.append(Q)
  temp = copy.deepcopy(P)
  for i in range(0, n - 1):
    components.append(np.dot(temp, Q))
    temp = np.dot(temp, P)
  S = np.hstack(components)
  return linalg.matrix_rank(S) == n

In [44]:
isControllable(P, Q, n)

True

In [45]:
t = symbols('t')

In [46]:
P = Matrix(P)
Q = Matrix(Q)
f = Matrix(f)
x0 = Matrix(x0)
x1 = Matrix(x1)

$$ Y = e^{Pt} $$

In [47]:
Y = exp(P*t)
Y

Matrix([
[exp(t), t*exp(t)],
[     0,   exp(t)]])

$$ B = Y(t)^{-1} \cdot Q(t) $$

In [48]:
B = Y.inv() * Q
B

Matrix([
[-t*exp(-t)],
[   exp(-t)]])

$$ A = \int_{0}^{T} B(t) \cdot B(t)^{t} dt $$

In [49]:
A = integrate(B * B.transpose(), (t, 0, T))
A

Matrix([
[ 1/4 - 5*exp(-2)/4, -1/4 + 3*exp(-2)/4],
[-1/4 + 3*exp(-2)/4,    1/2 - exp(-2)/2]])

$$\mu = Y(T)^{-1} \cdot x_{T} - x_{0} - \int_{0}^{T} Y(t)^{-1} \cdot f(t) dt$$

In [52]:
mu = Y.inv().subs(t, T) * x1 - x0 - integrate(Y.inv()*f, (t, 0, T))
mu

Matrix([
[      0],
[exp(-1)]])

Solving system: $$AC=\mu$$ $$C = A^{-1} \cdot \mu$$

In [54]:
C = A.inv() * mu
simplify(C)

Matrix([
[4*E*(-3 + exp(2))/(-6*exp(2) + 1 + exp(4))],
[4*E*(-5 + exp(2))/(-6*exp(2) + 1 + exp(4))]])

In [58]:
C.evalf()

Matrix([
[ 4.2368213933331],
[2.30619152767002]])

$$ u = B(t)^{T} \cdot C $$

In [59]:
u = B.transpose() * C
simplify(u)

Matrix([[4*(-t*exp(2) + 3*t - 5 + exp(2))*exp(1 - t)/(-6*exp(2) + 1 + exp(4))]])

# Program #2

$$ \{
\begin{aligned}
  \dot{x} &= Px + f(t)  \\
  y &= Rx + \phi(t)
\end{aligned} $$

$$ Given: y = \Phi(t) $$

$$ P ∈ [n \times n] \\ f(t) ∈ [n \times 1] \\ R(t) ∈ [m \times n] \\ g(t) ∈ [m \times 1] \\ \phi ∈ [m \times 1]  $$

In [81]:
n = 3
m = 1
P = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
f = np.array([[0], [0], [0]])
R = np.array([[0, 0, 1]])
phi = np.array([[0]])
Phi = np.array([[2]])
T = 1
x0 = 0
x1 = 1

In [82]:
def isObservable(R, P, n):
  components = []
  components.append(R.transpose())
  temp = copy.deepcopy(P.transpose())
  for i in range(0, n - 1):
    components.append(np.dot(temp, R.transpose()))
    temp = np.dot(temp, P.transpose())
  S = np.hstack(components)
  return linalg.matrix_rank(S) == n

In [83]:
isObservable(R, P, n)

True

In [84]:
t = symbols('t')

In [85]:
P = Matrix(P)
R = Matrix(R)

$$ Y = e^{Pt} $$

In [86]:
Y = exp(P*t)
Y

Matrix([
[     1, 0, 0],
[     t, 1, 0],
[t**2/2, t, 1]])

$$ H(t) = R \cdot Y(t) $$

In [87]:
H = R * Y
H

Matrix([[t**2/2, t, 1]])

$$ D = \int_{0}^{T} H(t)^{T} \cdot H(t) dt $$

In [88]:
D = integrate(H.transpose() * H, (t, 0, T))
D

Matrix([
[1/20, 1/8, 1/6],
[ 1/8, 1/3, 1/2],
[ 1/6, 1/2,   1]])

$$ g(t) = y(t) - \phi(t) - H(t) * \int_{0}^{t} Y(\tau)^{-1} \cdot H(\tau) d\tau $$

In [90]:
g = Phi - phi - H * integrate(Y.inv() * f, (t, 0, t))
g

Matrix([[2]])

$$ \mu = \int_{0}^{T} H(t)^{T} \cdot g(t) dt $$

In [91]:
mu = integrate(H.transpose() * g, (t, 0, T))
mu

Matrix([
[1/3],
[  1],
[  2]])

$$ D \cdot x_{0} = mu  $$

In [94]:
x_init = D.inv() * mu
x_init

Matrix([
[0],
[0],
[2]])

$$ x = Y(t) \cdot x_{0}$$

In [95]:
x_final = Y * x_init
x_final

Matrix([
[0],
[0],
[2]])